# preprocess

remeber to download 'PyMuPDF' befor running these codes <br> 
you can copy this code and paste it on your terminal: <br>
<em><strong>python -m pip install --upgrade pip <br>
python -m pip install PyMuPDF</strong> </em><br>
test if you really successfully install the package with the code below

In [2]:
import sys, site
print("sys.executable =", sys.executable)
print("site-packages =", site.getsitepackages())

import fitz
print("fitz file =", fitz.__file__)


sys.executable = c:\Users\owoyi\AppData\Local\Programs\Python\Python312\python.exe
site-packages = ['c:\\Users\\owoyi\\AppData\\Local\\Programs\\Python\\Python312', 'c:\\Users\\owoyi\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\site-packages']
fitz file = c:\Users\owoyi\AppData\Local\Programs\Python\Python312\Lib\site-packages\fitz\__init__.py


ensure the loop can read all the documents in the file

In [24]:
import os

folder_path = "C:/Users/owoyi/Downloads/盧教授/專利分析/data"

# List all files in the folder
files = os.listdir(folder_path)

# Print the names of all documents
for file in files:
    print(file)

cp01.pdf
cp02.pdf
cp03.pdf
cp04.pdf
cp05.pdf
cp06.pdf
cp07.pdf
cp08.pdf
cp09.pdf
cp10.pdf
cp11.pdf
cp12.pdf
cp13.pdf
cp14.pdf
cp15.pdf
cp16.pdf
cp17.pdf
cp18.pdf
cp19.pdf
cp20.pdf
cp21.pdf
cp22.pdf


to know how many cases are in this file <br>
usually there are 98-100 cases in a document 

In [15]:
import fitz
import re

def extract_target_number(pdf_path):
    doc = fitz.open(pdf_path)

    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        blocks = page.get_text("dict")["blocks"]

        for b in blocks:
            if "lines" in b:
                for line in b["lines"]:
                    for span in line["spans"]:
                        if (
                            'number' in b and b['number'] == 5
                            and 'size' in span and span['size'] == 12.0
                            and 'font' in span and span['font'] == 'Helvetica-Bold'
                            and page_num == 0  # Only on the first page
                        ):
                            match = re.search(r'\((\d+)\)', span["text"])
                            if match:
                                return int(match.group(1))

    doc.close()


In [16]:
# in this sample there are 84 case in the document
print(extract_target_number('C:/Users/owoyi/Downloads/盧教授/專利分析/data/cp22.pdf'))

84


all the cases have their own title <br>
usually they are in this format:  company_A v. company_B' <br>
this function used to get the title list

In [12]:
import fitz
import re

def extract_title(pdf_path):
    doc = fitz.open(pdf_path)
    extracted_text_list = []

    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        blocks = page.get_text("dict")["blocks"]

        for b in blocks:
            if "lines" in b:
                for line in b["lines"]:
                    for span in line["spans"]:
                        if 'font' in span and span['font'] == 'Helvetica-Oblique':
                            result_string = re.sub(r',[^,]*$', '', span["text"].strip())
                            extracted_text_list.append(result_string)

        if len(extracted_text_list) >= extract_target_number(pdf_path):
            break

    doc.close()
    return extracted_text_list


get list from the sample document

In [18]:

pdf_path = "C:/Users/owoyi/Downloads/盧教授/專利分析/data/cp22.pdf"
# pdf_path = "D:/antitrust/verdict/Files (100).PDF"
result_list = extract_title(pdf_path)

print out the list

In [19]:
result_list[:5]

['Golden v. Michael Grecco Prods.',
 'Motorola Solutions v. Hytera Communs. Corp.',
 'VHT, Inc. v. Zillow Grp., Inc.',
 'Adlife Mktg. & Communs. Co. v. June Media Inc.',
 'Abkco Music v. Sagan']

get other information and write them into csv

In [33]:
import fitz
import csv

def getAndWrite(pdf_path, case_list):
    doc = fitz.open(pdf_path)
    matching_texts = []
    titles_list = set()

    title = None  # Variable to store 'number': 1 text
    court = None  # Variable to store 'number': 2 text
    date = None  # Variable to store 'number': 3 text
    no = None  # Variable to store 'number': 4 text
    consider_number_2 = False  # Flag to indicate whether to consider 'number': 2 text
    
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        blocks = page.get_text("dict")["blocks"]
        for b in blocks:
            if 'number' in b and b['number'] == 1:
                if "lines" in b:
                    # Reset text variables for each new 'number': 1
                    title = ""
                    court = None
                    date = None
                    no = None

                    for line in b["lines"]:
                        for span in line["spans"]:
                            if 'font' in span and span['font'] == 'Helvetica-BoldOblique':
                                title = span["text"].strip()
                                titles_list.add(title)
                                for case_name in case_list:
                                    if title.strip() == case_name.strip():
                                        consider_number_2 = True
                                        break  # Stop looking for 'number': 1 text once found
            if consider_number_2 and 'number' in b and b['number'] == 2:
                if "lines" in b:
                    for line in b["lines"]:
                        for span in line["spans"]:
                            if ('font' in span and span['font'] == 'Helvetica') and ('size' in span and span['size'] == 10):
                                court = span["text"].strip()
            if consider_number_2 and 'number' in b and b['number'] == 3:
                if "lines" in b:
                    for line in b["lines"]:
                        for span in line["spans"]:
                            if ('font' in span and span['font'] == 'Helvetica') and ('size' in span and span['size'] == 10):
                                date = date.strip() + span["text"].strip() if date else span["text"].strip()
            if consider_number_2 and 'number' in b and b['number'] == 4:
                if "lines" in b:
                    for line in b["lines"]:
                        for span in line["spans"]:
                            if ('font' in span and span['font'] == 'Helvetica') and ('size' in span and span['size'] == 10):
                                no = span["text"].strip()
                                consider_number_2 = False
                                break  # Stop looking for 'number': 4 text once found
        # Check if 'number': 1, 'number': 2, 'number': 3, and 'number': 4 text are present, then append to the result
        if no:
            matching_texts.append({'pdf': pdf_path,'Title': title, 'Court': court, 'Date': date, 'No': no, 'page': page_num + 1 })

    doc.close()

    # Get the difference set 差集
    difference_set = set(case_list).symmetric_difference(titles_list)
    print(pdf_path)
    print(difference_set)
    print('======================================')
    return matching_texts
    


save in csv

In [ ]:
# Initialize CSV file outside the files loop
output_csv = "output.csv"
with open(output_csv, 'w', newline='', encoding='utf-8') as csv_file:
    fieldnames = ['pdf', 'Title', 'Court', 'Date', 'No', 'page']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()

for file in files:
    pdf_path = os.path.join(folder_path, file)
    case_list = extract_title(pdf_path)
    matching_texts = getAndWrite(pdf_path, case_list)
    
    # Append to the CSV file without overwriting it
    with open(output_csv, 'a', newline='', encoding='utf-8') as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        
        # Write the data
        for result in matching_texts:
            writer.writerow(result)


save in mongodb

In [ ]:
import os
from pymongo import MongoClient, UpdateOne

# --- MongoDB connection ---
MONGO_URI = "mongodb://*"  # <-- put your own mongo_uri here
DB_NAME = "copyright"
COL_NAME = "testing_writein"  # collection you would like to save in 

client = MongoClient(MONGO_URI)
db = client[DB_NAME]
col = db[COL_NAME]



for file in files:
    # Build the full path to the PDF file
    pdf_path = os.path.join(folder_path, file)

    # Extract the list of target case titles from the PDF
    case_list = extract_title(pdf_path)

    # Extract structured metadata (list of dicts)
    matching_texts = getAndWrite(pdf_path, case_list)

    # Skip this PDF if no matching cases are found
    if not matching_texts:
        continue

    # Collect bulk database operations for efficiency
    ops = []

    for r in matching_texts:
        # Safety check: ensure the PDF identifier exists
        # (only set if missing)
        r.setdefault("pdf", file)  # or use pdf_path if you prefer full paths
        # r.setdefault("pdf_path", pdf_path)  # uncomment if you want to store full paths

        # --- Option A: Simple insert (will create duplicates) ---
        # col.insert_many(matching_texts)

        # --- Option B (recommended): Upsert to avoid duplicates ---
        # Define the logical unique key for this record
        key = {
            "pdf": r.get("pdf"),
            "page": r.get("page"),
            "No": r.get("No")
        }

        # If a record with the same key exists, update it;
        # otherwise, insert a new document
        ops.append(
            UpdateOne(
                key,
                {"$set": r},   # overwrite fields with the latest extracted data
                upsert=True
            )
        )

    # Execute all database operations in a single bulk request
    if ops:
        col.bulk_write(ops, ordered=False)


print("Done.")


MuPDF error: library error: zlib error: unknown compression method

MuPDF error: library error: FT_New_Memory_Face(Arial): unknown file format

MuPDF error: library error: zlib error: unknown compression method

MuPDF error: library error: FT_New_Memory_Face(Arial): unknown file format

MuPDF error: library error: zlib error: unknown compression method

MuPDF error: library error: FT_New_Memory_Face(Arial): unknown file format

MuPDF error: library error: zlib error: unknown compression method

MuPDF error: library error: FT_New_Memory_Face(Arial): unknown file format

MuPDF error: library error: zlib error: unknown compression method

MuPDF error: library error: FT_New_Memory_Face(Arial): unknown file format

MuPDF error: library error: zlib error: unknown compression method

MuPDF error: library error: FT_New_Memory_Face(Arial): unknown file format

MuPDF error: library error: zlib error: unknown compression method

MuPDF error: library error: FT_New_Memory_Face(Arial): unknown file 

check and fix the title manually

In [31]:
# Example usage:
pdf_path = "C:/Users/owoyi/Downloads/盧教授/專利分析/data/cp22.pdf"
case_list = result_list
output_csv = "output_a.csv"
getAndWrite(pdf_path, case_list)

C:/Users/owoyi/Downloads/盧教授/專利分析/data/cp22.pdf
{'Barba v. Joy Travel Ny', 'Golden v. Michael Grecco Prods.', 'Wareka v. Faces', 'Tee Turtle, LLC v. Swartz', 'Jackson v. JPay, Inc.', 'awarding damages', 'Adlife Mktg. & Communs. Co. v. June Media Inc.', 'Panda Rest. Group v. Enymedia, Inc.', 'Pearson Educ., Inc. v. Labos', 'Safety Nailer LLC v. Individuals, P\'ships, & Unincorporated Ass’ns Identified on Schedule "A"', 'HN11', "West Stone Works Co. v. Wilson's Funeral Home", 'Neutral Gray Music v. Tri-City Funding & Mgmt. LLC and Smith', 'Boyd v. Does It Even Matter, LLC', "Animaccord Ltd. v. Individuals, P’ships & Unincorporated Ass'ns Identified on Schedule A", 'Healthplan Servs. v. Dixit', 'Zuffa, LLC v. Shin', 'Juliff v. Headout, Inc.', 'Bigelow v. Jerrick Ventures, LLC', 'RULE 7.1', 'Dist. LEXIS 248178', 'HN14', 'damages award', 'Abkco Music v. Sagan', 'Reilly v. Wozniak', 'Perea v. Editorial Cultural, Inc.', 'Crisman v. Van Der Hoog', 'Deyoung v. Wear', 'Skyrocket, LLC v. Comeyun'

[]

In [ ]:
# import os

# # Set the path of the directory
# folder_path = 'D:/antitrust/Data 1/data14'

# # Get a list of files in the directory
# files = os.listdir(folder_path)

# # Initialize a counter for the file names
# counter = 1

# for file_name in files:
#     # Get the file extension
#     extension = os.path.splitext(file_name)[1]
    
#     # Construct the new file name
#     new_name = f"d{counter}{extension}"
    
#     # Get the full file paths
#     old_file_path = os.path.join(folder_path, file_name)
#     new_file_path = os.path.join(folder_path, new_name)
    
#     # Rename the file
#     os.rename(old_file_path, new_file_path)
    
#     # Increment the counter
#     counter += 1

# print("Files have been renamed.")


Files have been renamed.


In [ ]:
# import pandas as pd

# # Read the CSV file into a DataFrame
# df = pd.read_csv('your_file.csv')

# # Display the first few rows of the DataFrame
# print(df.head())